# ML4CPS Project-2 | NB-4

In [ ]:
import os
import sys

basepath = os.path.abspath(os.path.join(".."))
if not basepath in sys.path:
    sys.path.append(basepath)

%load_ext autoreload
%autoreload 2

In [ ]:
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

In [ ]:
from utils.dataset import DatasetLoaderXL

## Load dataset

In [ ]:
dsxl = DatasetLoaderXL(dataset_dir="../dataset/raw", seed=42, verbose=True)
dsxl.load_all_datasets()

In [ ]:
dsxl.list_suburbs()

In [ ]:
dsxl.list_categories()

In [ ]:
dsxl.list_subcategories("Geography");

In [ ]:
subcategories = [
    "Population Density",
    "Travel time to GPO (minutes)",
    "Distance to GPO (km)",
    "Area (km^2)",
]
df = dsxl.get_subcategories_across_all_suburbs(subcategories)
df

In [ ]:
from sklearn.preprocessing import minmax_scale

df_norm = minmax_scale(df, axis=0)
df_norm = pd.DataFrame(df_norm, columns=df.columns, index=df.index)
df_norm

In [ ]:
correlation_matrix = df.corr()
correlation_matrix

In [ ]:
correlation_pairs = correlation_matrix.unstack().reset_index()
correlation_pairs.columns = ['Variable1', 'Variable2', 'Correlation']
correlation_pairs['AbsCorrelation'] = correlation_pairs['Correlation'].abs()
correlation_pairs = correlation_pairs[correlation_pairs['Variable1'] != correlation_pairs['Variable2']]
correlation_pairs = correlation_pairs.sort_values(by='AbsCorrelation', ascending=True)
correlation_pairs = correlation_pairs[correlation_pairs['Variable1'] < correlation_pairs['Variable2']]
correlation_pairs

In [ ]:
from scipy.stats import f_oneway

anova_results = {}
for subcategory in subcategories:
    anova_results[subcategory] = f_oneway(*[df[subcategory].values for suburb in df.index])
anova_results

In [ ]:
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 7))
dendrogram = sch.dendrogram(sch.linkage(df_norm, method='ward'))
plt.title('Dendrogram')
plt.xlabel('Suburbs')
plt.ylabel('Euclidean distances')
plt.show()

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity

In [ ]:
def get_similarity_matrix(df, metric):
    return pd.DataFrame(metric(df), index=df.index, columns=df.index)

In [ ]:
euclidean_similarity = lambda x: 1/(1 + euclidean_distances(x))

In [ ]:
dfs = get_similarity_matrix(df_norm, euclidean_similarity)
dfs

In [ ]:
dfs = get_similarity_matrix(df_norm, cosine_similarity)
dfs

In [ ]:
def get_similar_suburbs(similarity_matrix, n=5):
    similarity_matrix = similarity_matrix.copy()
    np.fill_diagonal(similarity_matrix.values, 0)
    return similarity_matrix.apply(lambda x: x.nlargest(n).index.tolist(), axis=1)

In [ ]:
get_similar_suburbs(dfs, 5)

In [ ]:
def run_mds_and_plot(similarity_matrix, n_components=2, random_state=42):
    mds = MDS(n_components=n_components, dissimilarity="precomputed", random_state=random_state)
    dissimilarity_matrix = 1 - similarity_matrix
    mds_results = mds.fit_transform(dissimilarity_matrix)
    mds_df = pd.DataFrame(mds_results, index=similarity_matrix.index, columns=[f"MDS{i+1}" for i in range(n_components)])

    plt.figure(figsize=(10, 7))
    sns.scatterplot(x="MDS1", y="MDS2", data=mds_df)
    for i in mds_df.index:
        plt.text(mds_df.loc[i, "MDS1"], mds_df.loc[i, "MDS2"], i, fontsize=9)
    plt.title('MDS Plot')
    plt.xlabel('MDS1')
    plt.ylabel('MDS2')
    plt.show()

    return mds_df

mds_df = run_mds_and_plot(dfs)

In [ ]:
dsxl.get_subcategory_across_all_suburbs("Location")

In [ ]:
import re
from math import radians, sin, cos, sqrt

def extract_coordinates(location):
    match = re.search(r'(\d+)km ([A-Z]+) of Melbourne', location)
    if match:
        distance = int(match.group(1))
        direction = match.group(2)
        angle = {
            'N': 0,
            'NNE': 22.5,
            'NE': 45,
            'ENE': 67.5,
            'E': 90,
            'ESE': 112.5,
            'SE': 135,
            'SSE': 157.5,
            'S': 180,
            'SSW': 202.5,
            'SW': 225,
            'WSW': 247.5,
            'W': 270,
            'WNW': 292.5,
            'NW': 315,
            'NNW': 337.5
        }[direction]
        return distance, angle
    return None, None

def polar_to_cartesian(distance, angle):
    angle_rad = radians(angle)
    x = distance * cos(angle_rad)
    y = distance * sin(angle_rad)
    return x, y

def calculate_proximity_matrix(coordinates):
    suburbs = list(coordinates.keys())
    proximity_matrix = pd.DataFrame(index=suburbs, columns=suburbs)
    for suburb1 in suburbs:
        for suburb2 in suburbs:
            x1, y1 = coordinates[suburb1]
            x2, y2 = coordinates[suburb2]
            distance = sqrt((x2 - x1)**2 + (y2 - y1)**2)
            proximity_matrix.loc[suburb1, suburb2] = distance
    return proximity_matrix

def get_coordinates():
    coordinates = {}
    location_df = dsxl.get_subcategory_across_all_suburbs("Location")
    for suburb, row in location_df.iterrows():
        distance, angle = extract_coordinates(row['Location'])
        if distance is not None and angle is not None:
            coordinates[suburb] = polar_to_cartesian(distance, angle)
    return coordinates

In [ ]:
coordinates = get_coordinates()
proximity_matrix = calculate_proximity_matrix(coordinates)
proximity_matrix

In [ ]:
def rotate_coordinates(coordinates, angle):
    angle_rad = radians(angle)
    rotated_coords = {}
    for suburb, (x, y) in coordinates.items():
        x_rot = x * cos(angle_rad) - y * sin(angle_rad)
        y_rot = x * sin(angle_rad) + y * cos(angle_rad)
        rotated_coords[suburb] = (x_rot, y_rot)
    return rotated_coords

def reflect_coordinates(coordinates):
    reflected_coords = {}
    for suburb, (x, y) in coordinates.items():
        reflected_coords[suburb] = (-x, -y)
    return reflected_coords

def reflect_coordinates2(coordinates):
    reflected_coords = {}
    for suburb, (x, y) in coordinates.items():
        reflected_coords[suburb] = (-x, y)
    return reflected_coords

In [ ]:
coordinates = get_coordinates()
coordinates = reflect_coordinates(coordinates)
coordinates = reflect_coordinates2(coordinates)
coordinates = rotate_coordinates(coordinates, 90)

plt.figure(figsize=(10, 10))
for suburb, (x, y) in coordinates.items():
    plt.scatter(x, y, label=suburb)
    plt.text(x, y, suburb, fontsize=9)

plt.title('Suburb Coordinates')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.grid(True)
# plt.legend(loc='best', bbox_to_anchor=(1, 1))
plt.show()